<a href="https://colab.research.google.com/github/reshmighosh/PowerSystems_ML/blob/development/Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# import libraries

import os
import numpy as np
import pandas as pd
import statsmodels.api
import statsmodels.graphics.tsaplots as tsa
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
!pip install netCDF4
import netCDF4 as nk
import glob

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 4.7MB 10.7MB/s 
     |████████████████████████████████| 317kB 66.8MB/s 


In [6]:
!pip install catboost

     |████████████████████████████████| 67.3MB 94kB/s 


In [109]:
import catboost
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import gradient_boosting
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# directory paths
path = '/content/drive/My Drive/Research/ELCC/Data_slurm/'
#os.chdir(path)
ercot_temp_path = 'ercot_temp/'
ercot_demand_path = 'ercot_demand/'
results_dir = 'Chp2/'

In [11]:
## function to import and concatenate temperature data


def importERCOTtemp(year, start_year, end_year, lat, lon):
    final = list()
    years = list(np.arange(start_year, end_year+1, 1))
    num_periods = end_year - start_year + 1
    #print(num_periods)
    
    final_temp = dict()
    for year_int in years:
        temp_file = nk.Dataset(path + ercot_temp_path + 'processedMERRAercot' + str(year_int) + '.nc')
        temp = temp_file.variables['T2M']
        powGen_lats = np.array(temp_file.variables['lat'][:])
        powGen_lons = np.array(temp_file.variables['lon'][:])
        lat_interest = np.where(np.abs(Dallas_lat - powGen_lats).argmin())
        #unique, counts = np.unique(np.array(temp), return_counts= True)
        #frequencies = np.asarray((unique, counts)).T
        #print(f'unique values for year {year} is {frequencies}')
        temp_celcius = np.array(temp) - 273 
        lat_interest = (np.abs(Dallas_lat - powGen_lats)).argmin()
        lon_interest = (np.abs(Dallas_lon - powGen_lons)).argmin()
        temp_final = temp_celcius[lat_interest *lon_interest, :, :].ravel()
        final.append(temp_final)
        final_temp[str(year)] = temp_final
    
    #large array
    final_array = np.concatenate(final, axis = 0)
    final_df = pd.DataFrame(final_array, columns = ['Temperature'])

    # converting final temperature to dataframe
    final_temp = pd.DataFrame(data = final_temp[str(year)], columns = ['Temperature'])
    time = pd.date_range('01-01-' + str(start_year), periods = 8760*num_periods, freq = 'H')

    final_df['Hour'] = time
    final_df.set_index('Hour', inplace = True)
    #final_temp['Hour'] = time
    #final_temp.set_index('Hour')
  

    return final_df#, final_temp

In [12]:
## Import temperature files from ERCOT
Dallas_lat = 32.7767
Dallas_lon = -96.7970
global years
temps = importERCOTtemp(2019, 2017, 2019, Dallas_lat, Dallas_lon)
temps


,Temperature
Hour,
2017-01-01 00:00:00,16.715424
2017-01-01 01:00:00,16.593781
2017-01-01 02:00:00,16.225159
2017-01-01 03:00:00,15.709961
2017-01-01 04:00:00,15.025146
...,...
2019-12-31 19:00:00,15.673767
2019-12-31 20:00:00,15.734222
2019-12-31 21:00:00,15.323975


In [71]:
def createRegDf(temps):
    #Create reg df w/ all data
    #demand  = load.copy()
    #demand.reset_index(inplace = True)
    #print(demand['NORTH_C'])
    regDf = temps.copy()
    #print(regDf.index)
    regDf['season'] = (regDf.index.month%12+3)//3 - 1
    #print("excecuted")
    regDf['year'] = regDf.index.year
    #print("year done")
    regDf['hour'] = regDf.index.hour
    #print("hour done")
    regDf['dayType'] = regDf.index.weekday//5 #weekend vs weekday; 5 is value of Saturday
    #print("day type done")
    #regDf['load'] = demand['NORTH_C']
    #print("demand added")
    #Combine day, hour, season into single series of fixed effects
    regDf['hourDaySeason'] = (regDf['hour'].astype(str) + regDf['dayType'].astype(str) + 
                                regDf['season'].astype(str))
    #regDf = pd.concat([regDf, demand['NORTH_C']], axis = 1)
    #regDf.dropna(inplace=True) #elim rows w/ missing load data
    return regDf

In [14]:
files = []
for dir, folder, file_path in os.walk(path + ercot_demand_path):
  for file in sorted(file_path):
    #if file.endswith('.xlsx') or file.endswith('.xls'):
    if file.endswith('.xlsx'):
      files.append(os.path.join(dir, file))
files = files[1:]
files


['/content/drive/My Drive/Research/ELCC/Data_slurm/ercot_demand/Native_Load_2017.xlsx',
 '/content/drive/My Drive/Research/ELCC/Data_slurm/ercot_demand/Native_Load_2018.xlsx',
 '/content/drive/My Drive/Research/ELCC/Data_slurm/ercot_demand/Native_Load_2019.xlsx',
 '/content/drive/My Drive/Research/ELCC/Data_slurm/ercot_demand/Native_Load_2020.xlsx']

In [15]:

def get_ERCOT_demand(files, start_year, end_year):
  n_periods = end_year - start_year + 1
  final_dfs = []
  for file in files:
    if file != path + ercot_demand_path + 'Native_Load_2020.xlsx':
      df = pd.read_excel(file)
      print(df.shape)
      if file == path + ercot_demand_path + 'Native_Load_2017.xlsx':
        df.iloc[7393, 0] = df.iloc[7393, 0][:-4]
      df.columns = ['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT']
      final_dfs.append(df)
    final_df = pd.concat(final_dfs, axis = 0)
    # treating hourly data timestamp
  #final_df.reset_index(inplace = True)
  final_df['Hour_End_new']= final_df['Hour_End'].str.replace('24:00', '00:00')
  final_df['Hour_End_new'] = pd.to_datetime(final_df['Hour_End_new'])
  final_df = final_df.drop('Hour_End',axis = 1)
  #time = pd.date_range('01-01-2003', periods = 8760*17, freq = 'H')
  #final_df['Time'] = time
  #print(final_df.shape)
  return final_df

In [91]:
demand = get_ERCOT_demand(files, 2017, 2019)
demand.columns

(8760, 10)
(8760, 10)
(8760, 10)


/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname DST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Index(['COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN', 'SOUTH_C',
       'WEST', 'ERCOT', 'Hour_End_new'],
      dtype='object')

In [62]:
#demand.set_index(['Hour_End_new'], inplace = True)
demand.index
demand.shape

(26280, 9)

In [72]:
final_df_reg = createRegDf(temps)
final_df_reg.index
final_df_reg.shape

(26280, 6)

In [88]:
demand = get_ERCOT_demand(files, 2017, 2019)
#demand_new_northC = demand['NORTH_C']
print(final_df_reg.shape, demand.shape)
final_df_reg = createRegDf(temps)
final_df_reg.reset_index(inplace = True)
demand = pd.DataFrame(data = np.arange(0, 26280, 1), columns = ['random'])
print(load.shape)
final_df_new = pd.concat([final_df_reg, demand], axis = 1)
final_df_new

(8760, 10)
(8760, 10)
(8760, 10)


/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname DST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


(26280, 7) (26280, 10)
(26280, 10)


,Hour,Temperature,season,year,hour,dayType,hourDaySeason,random
0,2017-01-01 00:00:00,16.715424,0,2017,0,1,010,0
1,2017-01-01 01:00:00,16.593781,0,2017,1,1,110,1
2,2017-01-01 02:00:00,16.225159,0,2017,2,1,210,2
3,2017-01-01 03:00:00,15.709961,0,2017,3,1,310,3
4,2017-01-01 04:00:00,15.025146,0,2017,4,1,410,4
...,...,...,...,...,...,...,...,...
26275,2019-12-31 19:00:00,15.673767,0,2019,19,0,1900,26275
26276,2019-12-31 20:00:00,15.734222,0,2019,20,0,2000,26276
26277,2019-12-31 21:00:00,15.323975,0,2019,21,0,2100,26277
26278,2019-12-31 22:00:00,13.566711,0,2019,22,0,2200,26278


In [93]:
demand['NORTH_C']


0        9239.153285
1        9104.997245
2        8988.035201
3        8979.148462
4        9033.547636
            ...     
8755    13164.660616
8756    12928.911509
8757    12716.274141
8758    12413.090126
8759    12084.775970
Name: NORTH_C, Length: 26280, dtype: float64

In [119]:
demand

,index,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT,Hour_End_new
0,0,01/01/2017 01:00,8791.789509,896.746302,1997.717635,683.621986,9239.153285,2366.632745,4490.781365,954.192864,29420.635691,2017-01-01 01:00:00
1,1,01/01/2017 02:00,8569.708419,865.930568,1997.781319,677.969375,9104.997245,2332.744630,4370.656830,951.025166,28870.813551,2017-01-01 02:00:00
2,2,01/01/2017 03:00,8326.425638,839.051175,1993.699160,671.998949,8988.035201,2237.506202,4210.650003,944.357749,28211.724078,2017-01-01 03:00:00
3,3,01/01/2017 04:00,8137.497400,822.829332,1995.540876,675.267971,8979.148462,2178.102265,4088.713039,943.188703,27820.288047,2017-01-01 04:00:00
4,4,01/01/2017 05:00,8011.869581,814.016188,1995.253501,663.619875,9033.547636,2133.953870,4021.757095,954.937932,27628.955677,2017-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
26275,8755,12/31/2019 20:00,11109.257856,1417.306742,3637.456602,825.761799,13164.660616,3181.359858,6453.697286,1290.258402,41079.759161,2019-12-31 20:00:00
26276,8756,12/31/2019 21:00,10846.500062,1411.761157,3639.019312,822.190573,12928.911509,3044.648834,6175.763027,1276.748646,40145.543121,2019-12-31 21:00:00
26277,8757,12/31/2019 22:00,10592.539874,1403.990584,3639.463899,819.394012,12716.274141,2930.563230,5968.287603,1260.087906,39330.601248,2019-12-31 22:00:00
26278,8758,12/31/2019 23:00,10304.624870,1387.051521,3630.184859,804.714775,12413.090126,2829.531899,5769.745220,1238.412254,38377.355525,2019-12-31 23:00:00


In [63]:
time = pd.date_range('01-01-2003', periods = 8760*17, freq = 'H')
time.shape

(148920,)

In [87]:
from datetime import timedelta
import pandas as pd
file1 = pd.read_excel(path + ercot_demand_path + 'Native_Load_2017.xlsx')
file1.columns = ['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT']
file1['Hour_End_new']= file1['Hour_End'].str.replace('24:00', '00:00')
file1['Hour_End_new'] = pd.to_datetime(file1['Hour_End_new'])
file1

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname DST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT,Hour_End_new
0,01/01/2017 01:00,8791.789509,896.746302,1997.717635,683.621986,9239.153285,2366.632745,4490.781365,954.192864,29420.635691,2017-01-01 01:00:00
1,01/01/2017 02:00,8569.708419,865.930568,1997.781319,677.969375,9104.997245,2332.744630,4370.656830,951.025166,28870.813551,2017-01-01 02:00:00
2,01/01/2017 03:00,8326.425638,839.051175,1993.699160,671.998949,8988.035201,2237.506202,4210.650003,944.357749,28211.724078,2017-01-01 03:00:00
3,01/01/2017 04:00,8137.497400,822.829332,1995.540876,675.267971,8979.148462,2178.102265,4088.713039,943.188703,27820.288047,2017-01-01 04:00:00
4,01/01/2017 05:00,8011.869581,814.016188,1995.253501,663.619875,9033.547636,2133.953870,4021.757095,954.937932,27628.955677,2017-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31/2017 20:00,12015.663549,1910.720587,2989.824568,1202.222117,20633.414345,3771.864865,9626.306799,1813.399697,53963.416528,2017-12-31 20:00:00
8756,12/31/2017 21:00,11883.114122,1907.585299,2970.854839,1191.309197,20174.983131,3719.315447,9519.748453,1795.416337,53162.326825,2017-12-31 21:00:00
8757,12/31/2017 22:00,11754.250889,1890.359350,2942.227121,1178.171245,19726.459865,3690.567365,9402.193277,1767.742081,52351.971192,2017-12-31 22:00:00
8758,12/31/2017 23:00,11579.853459,1855.937814,2898.567893,1157.488537,19211.158684,3689.751261,9260.416988,1735.764393,51388.939028,2017-12-31 23:00:00


In [83]:
file1

,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT,Hour_End_new
0,01/01/2017 01:00,8791.789509,896.746302,1997.717635,683.621986,9239.153285,2366.632745,4490.781365,954.192864,29420.635691,01/01/2017 01:00
1,01/01/2017 02:00,8569.708419,865.930568,1997.781319,677.969375,9104.997245,2332.744630,4370.656830,951.025166,28870.813551,01/01/2017 02:00
2,01/01/2017 03:00,8326.425638,839.051175,1993.699160,671.998949,8988.035201,2237.506202,4210.650003,944.357749,28211.724078,01/01/2017 03:00
3,01/01/2017 04:00,8137.497400,822.829332,1995.540876,675.267971,8979.148462,2178.102265,4088.713039,943.188703,27820.288047,01/01/2017 04:00
4,01/01/2017 05:00,8011.869581,814.016188,1995.253501,663.619875,9033.547636,2133.953870,4021.757095,954.937932,27628.955677,01/01/2017 05:00
...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31/2017 20:00,12015.663549,1910.720587,2989.824568,1202.222117,20633.414345,3771.864865,9626.306799,1813.399697,53963.416528,12/31/2017 20:00
8756,12/31/2017 21:00,11883.114122,1907.585299,2970.854839,1191.309197,20174.983131,3719.315447,9519.748453,1795.416337,53162.326825,12/31/2017 21:00
8757,12/31/2017 22:00,11754.250889,1890.359350,2942.227121,1178.171245,19726.459865,3690.567365,9402.193277,1767.742081,52351.971192,12/31/2017 22:00
8758,12/31/2017 23:00,11579.853459,1855.937814,2898.567893,1157.488537,19211.158684,3689.751261,9260.416988,1735.764393,51388.939028,12/31/2017 23:00


In [100]:
## Catboost
data = final_df_reg
labels = (demand['NORTH_C']).to_numpy().reshape(-1, 1)

X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42)


In [102]:
train_dataset = catboost.Pool(X_train, Y_train)
test_dataset = catboost.Pool(X_test, Y_test)

In [103]:
train_dataset

In [105]:
model = CatBoostRegressor(loss_function = 'RMSE')

In [106]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [107]:
model.grid_search(grid, train_dataset)

Streaming output truncated to the last 5000 lines.
151:	learn: 1425.3426852	test: 1421.2974116	best: 1421.2974116 (151)	total: 567ms	remaining: 179ms
152:	learn: 1423.4867634	test: 1419.1326048	best: 1419.1326048 (152)	total: 570ms	remaining: 175ms
153:	learn: 1420.8285076	test: 1415.7992439	best: 1415.7992439 (153)	total: 573ms	remaining: 171ms
154:	learn: 1417.9733509	test: 1412.9444756	best: 1412.9444756 (154)	total: 576ms	remaining: 167ms
155:	learn: 1416.3161425	test: 1411.1398069	best: 1411.1398069 (155)	total: 580ms	remaining: 164ms
156:	learn: 1414.1262733	test: 1408.7463219	best: 1408.7463219 (156)	total: 583ms	remaining: 160ms
157:	learn: 1412.0195402	test: 1406.5473826	best: 1406.5473826 (157)	total: 586ms	remaining: 156ms
158:	learn: 1410.4561582	test: 1404.7851130	best: 1404.7851130 (158)	total: 589ms	remaining: 152ms
159:	learn: 1407.4732852	test: 1401.6382307	best: 1401.6382307 (159)	total: 593ms	remaining: 148ms
160:	learn: 1406.0879143	test: 1400.0142004	best: 1400.014

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

In [111]:
# predicting on validation set

predictions = model.predict(X_test)
root_mse = (np.sqrt(mean_squared_error(Y_test, predictions)))

# Goodness of fit calculation
r2 = r2_score(Y_test, predictions)

# results
print('Testing performance')
print('RMSE: {:.2f}'.format(root_mse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 1040.04
R2: 0.92
